<a href="https://colab.research.google.com/github/heena2248/ImageClassification/blob/main/phd1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#paper 1

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Bidirectional, LSTM, Dense, Dropout, Flatten, Input, Reshape
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Load and preprocess the dataset
tp = pd.read_csv(tp.xls)
X = tp[:-1]
y = tp[-1]

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# One-hot encode the target variable
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y.reshape(-1, 1))

# Reshape X for Conv2D input
X = X.reshape(X.shape[0], 2, 2, 1)  # Assuming we reshape to (2, 2, 1) for demonstration purposes

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model architecture
input_layer = Input(shape=(2, 2, 1))

# Block 1
x = Conv2D(16, (1, 1), activation='relu', padding='same')(input_layer)
x = BatchNormalization()(x)
x = Conv2D(16, (1, 1), activation='relu', padding='same')(x)
x = Conv2D(16, (1, 2), activation='relu', padding='same')(x)

# Block 2
x = Conv2D(32, (1, 1), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(32, (1, 1), activation='relu', padding='same')(x)
x = Conv2D(32, (1, 2), activation='relu', padding='same')(x)

# Block 5
x = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(64, (1, 1), activation='relu', padding='same')(x)
x = Conv2D(64, (1, 2), activation='relu', padding='same')(x)

# Block 6
x = Conv2D(128, (1, 1), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, (1, 1), activation='relu', padding='same')(x)
x = Conv2D(128, (1, 2), activation='relu', padding='same')(x)

# Block 7
x = Conv2D(256, (1, 1), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (1, 1), activation='relu', padding='same')(x)
x = Conv2D(256, (1, 2), activation='relu', padding='same')(x)

# LSTM layer
x = Flatten()(x)
x = Reshape((1, -1))(x)  # Reshape for LSTM input
x = Bidirectional(LSTM(256))(x)

# Fully connected layer
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(3, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2, 2, 1)]         0         
                                                                 
 conv2d (Conv2D)             (None, 2, 2, 16)          32        
                                                                 
 batch_normalization (Batch  (None, 2, 2, 16)          64        
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 2, 2, 16)          272       
                                                                 
 conv2d_2 (Conv2D)           (None, 2, 2, 16)          528       
                                                                 
 conv2d_3 (Conv2D)           (None, 2, 2, 32)          544       
                                                             

In [ ]:
import random
import tensorflow as tf   # Needed for model cloning and training

# ===========================
# Coati Optimization Algorithm (COA)
# ===========================
# This is a meta-heuristic optimization algorithm inspired by natural behaviors
# It is used here to optimize hyperparameters of a deep learning model
# Similar in concept to Genetic Algorithms or Evolutionary Strategies

class CoatiOptimizationAlgorithm:
    def __init__(self, model, param_space, X_train, y_train, X_test, y_test, iterations=10, population_size=10):
        """
        Initialize the COA optimizer.

        Parameters:
        - model: A Keras model (untrained base model)
        - param_space: Dictionary of hyperparameters and their possible values
        - X_train, y_train: Training data
        - X_test, y_test: Testing data
        - iterations: Number of optimization iterations
        - population_size: Number of candidate solutions in each iteration
        """
        self.model = model
        self.param_space = param_space
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.iterations = iterations
        self.population_size = population_size

    # ------------------------------------
    def initialize_population(self):
        """
        Randomly initialize a population of candidate solutions.
        Each candidate = one set of hyperparameters.
        """
        population = []
        for _ in range(self.population_size):
            # Pick random values for each hyperparameter from the search space
            individual = {param: random.choice(values) for param, values in self.param_space.items()}
            population.append(individual)
        return population

    # ------------------------------------
    def fitness(self, individual):
        """
        Measure how 'good' a candidate solution is.
        Fitness = model accuracy on test data with that set of hyperparameters.
        """
        # Clone the base model so each individual trains independently
        model_copy = tf.keras.models.clone_model(self.model)
        # Compile with optimizer chosen from the individual (candidate)
        model_copy.compile(optimizer=individual['optimizer'],
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])
        # Train with chosen epochs and batch size
        model_copy.fit(self.X_train, self.y_train,
                       epochs=individual['epochs'],
                       batch_size=individual['batch_size'],
                       verbose=0)   # verbose=0 means no training logs

        # Evaluate accuracy on test set
        loss, accuracy = model_copy.evaluate(self.X_test, self.y_test, verbose=0)
        return accuracy   # Higher accuracy = better fitness

    # ------------------------------------
    def select_parents(self, population, fitnesses):
        """
        Select two parents based on their fitness (higher fitness = higher chance).
        Uses weighted random sampling.
        """
        parents = random.choices(population, weights=fitnesses, k=2)
        return parents

    # ------------------------------------
    def crossover(self, parent1, parent2):
        """
        Combine two parents to create a child solution.
        Each hyperparameter is randomly chosen from one of the parents.
        """
        child = {}
        for key in parent1.keys():
            # Child gets parameter from either parent1 or parent2
            child[key] = random.choice([parent1[key], parent2[key]])
        return child

    # ------------------------------------
    def mutate(self, individual):
        """
        Randomly change (mutate) one hyperparameter value.
        Mutation adds diversity and prevents early convergence.
        """
        mutation_key = random.choice(list(individual.keys()))  # Pick one hyperparameter
        individual[mutation_key] = random.choice(self.param_space[mutation_key])  # Replace with random value
        return individual

    # ------------------------------------
    def optimize(self):
        """
        Main optimization loop:
        1. Initialize population
        2. Evaluate fitness
        3. Select parents & generate children
        4. Apply crossover and mutation
        5. Repeat for given iterations
        6. Return best hyperparameters found
        """
        # Step 1: Create initial random population
        population = self.initialize_population()

        # Repeat optimization for given number of iterations
        for _ in range(self.iterations):
            # Step 2: Evaluate fitness for each individual
            fitnesses = [self.fitness(ind) for ind in population]

            # Step 3 & 4: Create next generation
            new_population = []
            for _ in range(self.population_size):
                # Select parents
                parents = self.select_parents(population, fitnesses)
                # Perform crossover
                child = self.crossover(parents[0], parents[1])
                # Small chance of mutation (10%)
                if random.random() < 0.1:
                    child = self.mutate(child)
                # Add new child to next generation
                new_population.append(child)

            # Step 5: Replace old population with new one
            population = new_population

        # Step 6: After all iterations, return the best solution
        fitnesses = [self.fitness(ind) for ind in population]
        best_individual = population[fitnesses.index(max(fitnesses))]
        return best_individual


# =====================================
# Example Usage
# =====================================

# Define parameter search space (hyperparameters to optimize)
param_space = {
    'optimizer': ['adam', 'sgd', 'rmsprop'],   # Choices of optimizers
    'epochs': [10, 20, 30],                    # Training epochs
    'batch_size': [16, 32, 64]                 # Batch sizes
}

# Initialize and run the Coati Optimization Algorithm
coa = CoatiOptimizationAlgorithm(model, param_space, X_train, y_train, X_test, y_test)

# Run optimization to find best hyperparameters
best_params = coa.optimize()

# Print best set of hyperparameters found
print("Best Parameters:", best_params)


Best Parameters: {'optimizer': 'sgd', 'epochs': 20, 'batch_size': 32}


In [ ]:
model.compile(optimizer=best_params['optimizer'], loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")

Epoch 1/20
4/4 [==============================] - 4s 15ms/step - loss: 1.0897 - accuracy: 0.3417
Epoch 2/20
4/4 [==============================] - 0s 15ms/step - loss: 1.0570 - accuracy: 0.4750
Epoch 3/20
4/4 [==============================] - 0s 15ms/step - loss: 1.0334 - accuracy: 0.4750
Epoch 4/20
4/4 [==============================] - 0s 16ms/step - loss: 1.0140 - accuracy: 0.4750
Epoch 5/20
4/4 [==============================] - 0s 15ms/step - loss: 0.9843 - accuracy: 0.6083
Epoch 6/20
4/4 [==============================] - 0s 15ms/step - loss: 0.9379 - accuracy: 0.7167
Epoch 7/20
4/4 [==============================] - 0s 15ms/step - loss: 0.9078 - accuracy: 0.7917
Epoch 8/20
4/4 [==============================] - 0s 15ms/step - loss: 0.8824 - accuracy: 0.8000
Epoch 9/20
4/4 [==============================] - 0s 15ms/step - loss: 0.8466 - accuracy: 0.8750
Epoch 10/20
4/4 [==============================] - 0s 15ms/step - loss: 0.8126 - accuracy: 0.8833
Epoch 11/20
4/4 [============

In [ ]:
#paper 2

In [ ]:
# ===============================
# Imports
# ===============================
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf


# ===============================
# Step 1: Data Preprocessing
# ===============================

# Standardize the features (mean=0, std=1) so that all features are on the same scale.
# This helps neural networks train faster and avoid bias from large values.
scaler = StandardScaler()
X = scaler.fit_transform(X)

# One-hot encode the target variable (labels).
# Example: if classes are [0,1,2], this will convert:
# 0 → [1,0,0], 1 → [0,1,0], 2 → [0,0,1]
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y.reshape(-1, 1))

# Split the dataset into training (80%) and testing (20%).
# random_state=42 ensures reproducibility.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


# ===============================
# Step 2: Define ESWO Algorithm
# ===============================
class ESWO:
    """
    Simplified implementation of the ESWO (Evolutionary Swarm Optimization) algorithm
    for feature selection using a neural network as the fitness evaluator.
    """

    def __init__(self, n_agents, n_iterations, data, target):
        """
        Initialize ESWO.

        Parameters:
        n_agents     : number of agents (candidate solutions)
        n_iterations : number of optimization iterations
        data         : feature matrix (X)
        target       : labels (y, one-hot encoded)
        """
        self.n_agents = n_agents
        self.n_iterations = n_iterations
        self.data = data
        self.target = target
        self.dim = data.shape[1]  # number of features (dimensions)

    def fitness(self, position):
        """
        Fitness function = model accuracy with selected features.

        Parameters:
        position : binary-like vector (values ~ [0,1]) where >0.5 means "select feature".

        Returns:
        accuracy of a simple neural network trained with the selected features.
        """
        # Convert continuous [0,1] positions into binary selection (0 or 1).
        selected_features = np.where(position > 0.5, 1, 0)

        # If no features are selected, return 0 accuracy.
        if np.sum(selected_features) == 0:
            return 0

        # Subset dataset to only selected features.
        selected_data = self.data[:, selected_features == 1]

        # Define a simple neural network model.
        model = tf.keras.Sequential([
            Dense(64, activation='relu', input_shape=(selected_data.shape[1],)),  # Input layer
            Dense(32, activation='relu'),                                        # Hidden layer
            Dense(3, activation='softmax')                                       # Output (3 classes)
        ])

        # Compile with Adam optimizer and categorical crossentropy loss.
        model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        # Train for 10 epochs (short training to evaluate quickly).
        model.fit(selected_data, self.target, epochs=10, batch_size=16, verbose=0)

        # Evaluate accuracy on the same data (note: no validation split here).
        loss, accuracy = model.evaluate(selected_data, self.target, verbose=0)

        return accuracy

    def optimize(self):
        """
        Run the optimization process.

        Returns:
        best feature subset as a binary mask (0 = not selected, 1 = selected).
        """
        # Step 1: Initialize agents randomly in [0,1].
        agents = np.random.rand(self.n_agents, self.dim)

        # Step 2: Evaluate the first agent as baseline.
        best_agent = agents[0]
        best_fitness = self.fitness(best_agent)

        # Step 3: Iterate over generations.
        for iteration in range(self.n_iterations):
            for i in range(self.n_agents):

                # Generate a new candidate by adding random noise (Gaussian).
                new_position = np.clip(
                    agents[i] + np.random.normal(0, 0.1, self.dim), 0, 1
                )

                # Evaluate its fitness (accuracy).
                new_fitness = self.fitness(new_position)

                # If better, update best solution.
                if new_fitness > best_fitness:
                    best_fitness = new_fitness
                    best_agent = new_position

                # Replace agent with new position (exploration step).
                agents[i] = new_position

        # Convert best_agent to binary mask (0/1).
        return np.where(best_agent > 0.5, 1, 0)


# ===============================
# Step 3: Run ESWO Optimization
# ===============================

# Initialize ESWO with 10 agents and 20 iterations.
eswo = ESWO(n_agents=10, n_iterations=20, data=X_train, target=y_train)

# Run optimization → returns binary mask of selected features.
selected_features = eswo.optimize()
print("Selected Features:", selected_features)


# ===============================
# Step 4: Reduce Dataset
# ===============================

# Keep only selected features for training and testing.
X_train_selected = X_train[:, selected_features == 1]
X_test_selected = X_test[:, selected_features == 1]

print("Reduced training set shape:", X_train_selected.shape)
print("Reduced test set shape:", X_test_selected.shape)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Selected Features: [1 0 1 1]


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Conv2D, DepthwiseConv2D, Input, Flatten

# =====================================================
# STEP 1: Define Cp-Res-DWSCNN Model
# =====================================================
def build_cp_res_dwscnn(input_shape):
    """
    Build a Cp-Res-DWSCNN model:
    - Depthwise separable convolution block
    - Residual connections
    - Fully connected layers
    """

    inputs = Input(shape=input_shape)

    # Depthwise Separable Convolution Block (cheap & efficient convolution)
    x = DepthwiseConv2D((1, 1), padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(32, (1, 1), padding='same', activation='relu')(x)

    # Residual Block (skip connection)
    shortcut = x  # Save the input for later addition
    x = Conv2D(32, (1, 1), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(32, (1, 1), padding='same', activation='relu')(x)
    x = tf.keras.layers.Add()([x, shortcut])  # Add skip connection

    # Flatten + Fully Connected Layers
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)  # Dropout for regularization
    outputs = Dense(3, activation='softmax')(x)  # Softmax for multi-class classification

    model = Model(inputs, outputs)
    return model


# =====================================================
# STEP 2: Define Hyperparameter Search Space
# =====================================================
param_space = {
    'optimizer': ['adam', 'rmsprop'],  # Optimizers to test
    'epochs': [5, 10, 12],             # Number of training epochs
    'batch_size': [16, 32, 64]         # Batch sizes
}


# =====================================================
# STEP 3: Define Fitness Function (to evaluate each candidate)
# =====================================================
def fitness_function(params):
    """
    Train Cp-Res-DWSCNN with given parameters and return accuracy on test data.
    """
    model = build_cp_res_dwscnn(input_shape=(X_train.shape[1], 1, 1))

    # Compile with selected optimizer
    model.compile(optimizer=params['optimizer'],
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Train model (reshape data to fit Conv2D input)
    model.fit(X_train.reshape(-1, X_train.shape[1], 1, 1),
              y_train,
              epochs=params['epochs'],
              batch_size=params['batch_size'],
              verbose=0)

    # Evaluate on test data
    loss, accuracy = model.evaluate(X_test.reshape(-1, X_test.shape[1], 1, 1),
                                    y_test,
                                    verbose=0)
    return accuracy


# =====================================================
# STEP 4: Base Optimizer Class
# =====================================================
class Optimizer:
    def __init__(self, param_space, fitness_function, population_size=10, iterations=20):
        self.param_space = param_space
        self.fitness_function = fitness_function
        self.population_size = population_size
        self.iterations = iterations

    def optimize(self):
        raise NotImplementedError  # Must be implemented in subclass


# =====================================================
# STEP 5: Pufferfish Optimization Algorithm (PFOA)
# =====================================================
class PufferfishOptimizationAlgorithm(Optimizer):

    def initialize_population(self):
        """
        Randomly initialize population of candidate hyperparameter sets.
        """
        population = [{key: np.random.choice(values)
                       for key, values in self.param_space.items()}
                      for _ in range(self.population_size)]
        return population

    def evaluate_population(self, population):
        """
        Evaluate each candidate using fitness function.
        """
        fitness_scores = [self.fitness_function(individual) for individual in population]
        return fitness_scores

    def inflate_and_deflate(self, individual, best_individual):
        """
        Mimic pufferfish behavior:
        - Inflate = try new random values for some parameters
        - Deflate = move closer to best individual found so far
        """
        # Inflate step: explore new values randomly
        inflated = {key: np.random.choice(self.param_space[key])
                    if np.random.rand() > 0.5 else individual[key]
                    for key in self.param_space}

        # Deflate step: exploit by moving toward best individual
        deflated = {key: best_individual[key]
                    if np.random.rand() > 0.5 else individual[key]
                    for key in self.param_space}

        return inflated, deflated

    def optimize(self):
        """
        Main optimization loop:
        1. Initialize population
        2. Evaluate fitness
        3. Iteratively update candidates via inflate/deflate
        4. Track best solution
        """
        # Step 1: Initialize population
        population = self.initialize_population()

        # Step 2: Evaluate initial population
        fitness_scores = self.evaluate_population(population)
        best_individual = population[np.argmax(fitness_scores)]
        best_fitness = max(fitness_scores)

        # Step 3: Optimization loop
        for iteration in range(self.iterations):
            new_population = []
            for i in range(self.population_size):
                # Generate new candidates (inflate & deflate)
                inflated, deflated = self.inflate_and_deflate(population[i], best_individual)
                new_population.extend([inflated, deflated])  # Add both

            # Keep only required population size
            new_population = new_population[:self.population_size]

            # Evaluate new population
            new_fitness_scores = self.evaluate_population(new_population)

            # Update best candidate
            best_candidate = new_population[np.argmax(new_fitness_scores)]
            best_candidate_fitness = max(new_fitness_scores)

            if best_candidate_fitness > best_fitness:
                best_fitness = best_candidate_fitness
                best_individual = best_candidate

            # Replace old population with new
            population = new_population

        return best_individual, best_fitness


# =====================================================
# STEP 6: Run PFOA Optimization
# =====================================================
optimizer = PufferfishOptimizationAlgorithm(param_space, fitness_function)

# Run optimizer
best_params, best_fitness = optimizer.optimize()

print("Best Parameters:", best_params)
print("Best Fitness (accuracy):", best_fitness)


# =====================================================
# STEP 7: Train Final Model with Best Hyperparameters
# =====================================================
model = build_cp_res_dwscnn(input_shape=(X_train.shape[1], 1, 1))
model.compile(optimizer=best_params['optimizer'],
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train final model with selected features
model.fit(X_train.reshape(-1, X_train.shape[1], 1, 1),
          y_train,
          epochs=best_params['epochs'],
          batch_size=best_params['batch_size'],
          verbose=1)

# Evaluate final model on test data
loss, accuracy = model.evaluate(X_test.reshape(-1, X_test.shape[1], 1, 1), y_test)
print(f"Test Accuracy: {accuracy}")


Best Parameters: {'optimizer': 'rmsprop', 'epochs': 5, 'batch_size': 64}
Best Fitness: 0.8333333134651184
Epoch 1/5
2/2 [==============================] - 1s 7ms/step - loss: 0.9486 - accuracy: 0.5000
Epoch 2/5
2/2 [==============================] - 0s 5ms/step - loss: 0.5284 - accuracy: 0.7833
Epoch 3/5
2/2 [==============================] - 0s 6ms/step - loss: 0.4919 - accuracy: 0.7500
Epoch 4/5
2/2 [==============================] - 0s 5ms/step - loss: 0.4379 - accuracy: 0.7750
Epoch 5/5
1/1 [==============================] - 0s 173ms/step - loss: 0.8955 - accuracy: 0.6333
Test Accuracy: 0.6333333253860474


In [ ]:
#paper 3

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Flatten, Input, Dense, Dropout, GlobalAveragePooling2D, Multiply
from tensorflow.keras.models import Model
import numpy as np
import random
from sklearn.decomposition import PCA

# -----------------------------------------------------------
# 1. STACKED CONVOLUTIONAL AUTOENCODER (SCAE)
# -----------------------------------------------------------

def build_scae(input_shape):
    """
    Build a simple stacked convolutional autoencoder.
    - Encoder compresses input into a smaller representation.
    - Decoder reconstructs the original input.
    """
    inputs = Input(shape=input_shape)

    # Encoder: progressively reduces channels
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = Conv2D(4, (3, 3), activation='relu', padding='same')(x)

    # Decoder: reconstructs input
    x = Conv2DTranspose(8, (3, 3), activation='relu', padding='same')(encoded)
    x = Conv2DTranspose(16, (3, 3), activation='relu', padding='same')(x)
    decoded = Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

    # Encoder model (for feature extraction)
    encoder = Model(inputs, encoded)
    # Autoencoder model (for training reconstruction)
    autoencoder = Model(inputs, decoded)

    return encoder, autoencoder


# Shape = (#features, 1, 1) → treat each feature like a pixel
input_shape = (X_train.shape[1], 1, 1)
encoder, autoencoder = build_scae(input_shape)

# Compile and train autoencoder to reconstruct inputs
autoencoder.compile(optimizer='adam', loss='mse')
X_train_reshaped = X_train.reshape(-1, X_train.shape[1], 1, 1)
X_test_reshaped = X_test.reshape(-1, X_test.shape[1], 1, 1)

autoencoder.fit(
    X_train_reshaped, X_train_reshaped,
    epochs=50, batch_size=16,
    validation_data=(X_test_reshaped, X_test_reshaped)
)

# Extract encoded features from the trained encoder
X_train_encoded = encoder.predict(X_train_reshaped)
X_test_encoded = encoder.predict(X_test_reshaped)

# Flatten encoded features into 2D shape
X_train_encoded = X_train_encoded.reshape(X_train_encoded.shape[0], -1)
X_test_encoded = X_test_encoded.reshape(X_test_encoded.shape[0], -1)

# -----------------------------------------------------------
# 2. PCA FEATURE REDUCTION
# -----------------------------------------------------------

# Apply PCA to reduce dimensionality
pca = PCA(n_components=3)  # keep top 3 principal components
X_train_pca = pca.fit_transform(X_train_encoded)
X_test_pca = pca.transform(X_test_encoded)

# Reshape PCA features for CNN input (like 1D signals)
input_shape_sednet = (X_train_pca.shape[1], 1, 1)
X_train_pca_reshaped = X_train_pca.reshape(-1, X_train_pca.shape[1], 1, 1)
X_test_pca_reshaped = X_test_pca.reshape(-1, X_test_pca.shape[1], 1, 1)

# -----------------------------------------------------------
# 3. SE-DENSE NET (SEDNET) CLASSIFIER
# -----------------------------------------------------------

def squeeze_excite_block(inputs, ratio=16):
    """
    Channel attention mechanism:
    - Squeezes global information into a vector.
    - Learns importance weights for each channel.
    - Excites (scales) the input channels accordingly.
    """
    filters = inputs.shape[-1]  # number of channels
    se = GlobalAveragePooling2D()(inputs)  # squeeze
    se = Dense(filters // ratio, activation='relu')(se)
    se = Dense(filters, activation='sigmoid')(se)  # excitation
    return Multiply()([inputs, se])


def build_sednet(input_shape):
    """
    Build SedNet classifier:
    - Several convolutional layers with squeeze-excite blocks.
    - Fully connected classifier at the end.
    """
    inputs = Input(shape=input_shape)

    # Conv layers with SE attention
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = squeeze_excite_block(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = squeeze_excite_block(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = squeeze_excite_block(x)

    # Flatten + FC classifier
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(3, activation='softmax')(x)  # 3-class classification

    return Model(inputs, outputs)

# -----------------------------------------------------------
# 4. CLOUDED LEOPARD OPTIMIZATION ALGORITHM (CLOA)
# -----------------------------------------------------------

class CloudedLeopardOptimization:
    """
    A metaheuristic optimization algorithm inspired by clouded leopard hunting strategy.
    Explores and exploits hyperparameter search space.
    """

    def __init__(self, param_space, fitness_function, population_size=10, iterations=20):
        self.param_space = param_space
        self.fitness_function = fitness_function
        self.population_size = population_size
        self.iterations = iterations

    def initialize_population(self):
        """Randomly sample hyperparameter combinations from param_space."""
        population = [
            {key: random.choice(values) for key, values in self.param_space.items()}
            for _ in range(self.population_size)
        ]
        return population

    def optimize(self):
        """Run CLOA optimization loop."""
        population = self.initialize_population()
        best_individual = population[0]
        best_fitness = self.fitness_function(best_individual)

        for iteration in range(self.iterations):
            new_population = []

            # Exploration: test new candidates
            for individual in population:
                new_individual = self.exploration_phase(individual)
                new_fitness = self.fitness_function(new_individual)
                if new_fitness > best_fitness:  # keep track of best
                    best_fitness = new_fitness
                    best_individual = new_individual
                new_population.append(new_individual)

            # Exploitation: refine around the best solution
            population = self.exploitation_phase(new_population, best_individual)

        return best_individual, best_fitness

    def exploration_phase(self, individual):
        """Randomly explore the space by resampling params."""
        return {key: random.choice(self.param_space[key]) for key in self.param_space}

    def exploitation_phase(self, population, best_individual):
        """Refine population by moving some solutions closer to the best one."""
        for i in range(len(population)):
            if random.random() < 0.5:
                # Replace individual completely with best
                population[i] = best_individual.copy()
            else:
                # Partially copy best params
                for key in self.param_space:
                    if random.random() < 0.5:
                        population[i][key] = best_individual[key]
        return population


# -----------------------------------------------------------
# 5. FITNESS FUNCTION (with FIX applied)
# -----------------------------------------------------------

# Hyperparameter search space
param_space = {
    'optimizer': ['adam', 'rmsprop'],
    'epochs': [5, 10, 12],
    'batch_size': [16, 32, 64]
}

def fitness_function(params):
    """
    Evaluate model performance for a given set of hyperparameters.
    FIX: Rebuilds SedNet model fresh each time so training is independent.
    """
    # Build a new model for every candidate
    model = build_sednet(input_shape_sednet)

    # Compile with chosen optimizer
    model.compile(optimizer=params['optimizer'],
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Train with candidate epochs and batch size
    model.fit(X_train_pca_reshaped, y_train,
              epochs=params['epochs'],
              batch_size=params['batch_size'],
              verbose=0)

    # Evaluate on test data
    _, accuracy = model.evaluate(X_test_pca_reshaped, y_test, verbose=0)

    return accuracy


# -----------------------------------------------------------
# 6. RUN OPTIMIZATION
# -----------------------------------------------------------

cloa = CloudedLeopardOptimization(param_space, fitness_function)
best_params, best_fitness = cloa.optimize()

print("Best Parameters:", best_params)
print("Best Fitness:", best_fitness)

# -----------------------------------------------------------
# 7. TRAIN FINAL MODEL WITH BEST PARAMS
# -----------------------------------------------------------

final_model = build_sednet(input_shape_sednet)
final_model.compile(optimizer=best_params['optimizer'],
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

final_model.fit(X_train_pca_reshaped, y_train,
                epochs=best_params['epochs'],
                batch_size=best_params['batch_size'],
                verbose=1)

# Final evaluation
loss, accuracy = final_model.evaluate(X_test_pca_reshaped, y_test)
print(f"Test Accuracy: {accuracy}")


Epoch 1/50
8/8 [==============================] - 1s 30ms/step - loss: 1.2666 - val_loss: 1.1674
Epoch 2/50
8/8 [==============================] - 0s 7ms/step - loss: 1.2591 - val_loss: 1.1587
Epoch 3/50
8/8 [==============================] - 0s 7ms/step - loss: 1.2464 - val_loss: 1.1447
Epoch 4/50
8/8 [==============================] - 0s 7ms/step - loss: 1.2252 - val_loss: 1.1212
Epoch 5/50
8/8 [==============================] - 0s 7ms/step - loss: 1.1944 - val_loss: 1.0822
Epoch 6/50
8/8 [==============================] - 0s 7ms/step - loss: 1.1404 - val_loss: 1.0212
Epoch 7/50
8/8 [==============================] - 0s 7ms/step - loss: 1.0585 - val_loss: 0.9385
Epoch 8/50
8/8 [==============================] - 0s 8ms/step - loss: 0.9643 - val_loss: 0.8628
Epoch 9/50
8/8 [==============================] - 0s 7ms/step - loss: 0.8940 - val_loss: 0.8173
Epoch 10/50
8/8 [==============================] - 0s 7ms/step - loss: 0.8606 - val_loss: 0.7910
Epoch 11/50
8/8 [=====================